<b>Writing views.py</b>
=======================

视图函数或视图，简而言之就是一个Python函数，它接受一个Web请求并返回一个Web响应。 此响应可以是网页的HTML内容，重定向或404错误，XML文档或图像...等，或者其他什么。视图本身包含各种逻辑，它是反回响应所必需的。该函数中的这段代码可以保存在任何你想要的地方，只要这个位置在你的Python路径上，没有其他要求。惯例是将视图放置在名为views.py的文件中，将这个文件放置在您的项目或应用程序目录中。

# 解析一个简单的view

下面是一个简单的view，它返回当前的日期和时间，作为一个HTML文档：

```
from django.http import HttpResponse
import datetime

def current_datetime(request):
    now = datetime.datetime.now()
    html = "<html><body>It is now %s.</body></html>" % now
    return HttpResponse(html)
```

首先，从`django.http`模块导入类`HttpResponse`，同时还导入Python的`datetime`库。

接下来，定义一个名为`current_datetime`的函数。这是一个`view`函数。每个视图函数都将`HttpRequest`对象作为其第一个参数，通常将其命名为`request`。

请注意，视图函数的名称无关紧要，只要符合Python函数命名规则即可。在这里将它命名为`current_datetime`，因为该名称清楚地表明它的作用。

该视图返回一个包含生成响应的`HttpResponse`对象。 每个视图函数负责返回一个`HttpResponse`对象。（有例外情况，稍后讨论。）

**注意：`django`有一个`TIME_ZONE`设置项，默认设置为`America/Chicago`，要正确地返回当地的时间，需要在项目的`settings.py`文件中修改该选项为当地的时区，比如`UTC-8`时区设置为`TIME_ZONE = 'Asia/Shanghai'`。

# 将 URLs 映射到 views


这个视图函数返回一个包含当前日期和时间的HTML页面。要在特定的URL上显示此视图，还需要创建一个URLconf; 有关说明，请参阅 [URL调度程序](URL dispatcher.ipynb)。

# 返回错误
在Django中返回HTTP错误代码很容易。 HttpResponse的子类可以用于许多不同于200的常见的HTTP状态码（这意味着“OK”）。您可以在`request/response`文档中找到可用子类的完整列表。只需返回其中一个子类的实例以表示错误，而不是返回常规的HttpResponse。 例如：

```
from django.http import HttpResponse, HttpResponseNotFound

def my_view(request):
    # ...
    if foo:
        return HttpResponseNotFound('<h1>Page not found</h1>')
    else:
        return HttpResponse('<h1>Page was found</h1>')
```

对于每个可能的HTTP响应代码，都没有专门的子类，因为其中很多不会很常见。但是，如HttpResponse文档中所述，您可以将HTTP状态代码传递给HttpResponse的构造函数，以便为您喜欢的任何状态代码创建返回类。 例如：

```
from django.http import HttpResponse

def my_view(request):
    # ...

    # Return a "created" (201) response code.
    return HttpResponse(status=201)
```

因为404错误是迄今为止最常见的HTTP错误，所以有一种更简单的方法来处理这些错误。

# Http404异常

**class** `django.http.Http404`

当你返回一个错误，如HttpResponseNotFound，你负责定义错误页面的HTML：

`return HttpResponseNotFound('<h1>Page not found</h1>')`

为了方便起见，并且由于在您的站点中存在一致的404错误页面是一个好主意，Django提供了一个Http404异常。 如果你在视图函数中的任何一点都提出Http404，Django会捕获它并返回你的应用程序的标准错误页面以及HTTP错误代码404。

**用法示例：**

```
from django.http import Http404
from django.shortcuts import render
from polls.models import Poll

def detail(request, poll_id):
    try:
        p = Poll.objects.get(pk=poll_id)
    except Poll.DoesNotExist:
        raise Http404("Poll does not exist")
    return render(request, 'polls/detail.html', {'poll': p})
    ```

为了在Django返回404时显示自定义HTML，您可以创建一个名为404.html的HTML模板，并将其放置在模板树的顶层。 此模板将在DEBUG设置为False时提供。

当DEBUG为True时，您可以提供一条消息给Http404，它将出现在标准的404调试模板中。将这些消息用于调试目的; 它们通常不适合用于生产404模板。

# 自定义错误视图
Django中的默认错误视图应该足以满足大多数Web应用程序的需求，但是如果您需要任何自定义行为，则可以轻松覆盖它们。 只需在URLconf中指定处理程序，如下所示（在其他地方设置它们将不起作用）。

**`page_not_found（）`视图被`handler404`覆盖：**

`handler404 = 'mysite.views.my_custom_page_not_found_view'

**`server_error()`视图被`handler500`:**

`handler500 = 'mysite.views.my_custom_error_view'`

**permission_denied()`视图被`handler403`:**

`handler403 = 'mysite.views.my_custom_permission_denied_view'`

**`bad_request()`视图被 `handler400`:**

`handler400 = 'mysite.views.my_custom_bad_request_view'`

**参考**

使用`CSRF_FAILURE_VIEW`设置重写`CSRF`错误视图。

**`CSRF_FAILURE_VIEW`**

默认值: `'django.views.csrf.csrf_failure'`

当传入的请求被CSRF保护拒绝时，视图函数将使用点运算路径。该函数应该有这个签名：

```
def csrf_failure(request, reason=""):
    ...
```
原因是一条简短消息（旨在用于开发人员或日志记录，而不是最终用户），说明请求被拒绝的原因。 它应该返回一个`HttpResponseForbidden`。

`django.views.csrf.csrf_failure（）`接受默认为`'403_csrf.html'`的其他`template_name`参数。 如果存在具有该名称的模板，它将用于呈现页面。